# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти один из двух тарифов "*Смарт*" и "*Ультра*" (из проекта курса «Статистический анализ данных»). Необходимо построить модель для задачи классификации, которая выберет подходящий новый тариф для абонентов, пользующихся старыми тарифами. 

Необходимо построить модель с максимально большим значением `accuracy` (доля правильных ответов должна составлять по крайней мере 0.75).

## 1. Открытие и изучение файла

In [ ]:
pip unistall pypiwin32

Сперва импортируем все библиотеки, которые понадобятся нам в дальнейшей работе.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from time import sleep
from tqdm import trange
from tqdm.notebook import tqdm

Создадим датафрейм **cell_comm** (от англ. cellular communication - "*сотовая связь*") и поместим в него набор данных.

In [2]:
cell_comm = pd.read_csv('C:\\Users\\503so\\OneDrive\\Desktop\\praktikum-to-git\\06_users_behavior.csv')

Запросим информацию о датафрейме.

In [3]:
cell_comm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Также выведем первые 5 строк таблицы.

In [4]:
cell_comm.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Датафрейм содержит 3214 строк и 5 столбцов:
   - сalls — количество звонков,
   - minutes — суммарная длительность звонков в минутах,
   - messages — количество sms-сообщений,
   - mb_used — израсходованный интернет-трафик в Мб,
   - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Проверим датафрейм на наличие в нем дубликатов.

In [6]:
cell_comm.duplicated().sum()

0

Дубликаты отсутствуют.


В типах данных также нет ошибок, однако можно привести показатели количества к целочисленному значению.

In [7]:
cell_comm[['calls', 'messages']] = cell_comm[['calls', 'messages']].astype(int)

Также взглянем на описание статистик датафрейма.

In [8]:
cell_comm.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<a id='1'></a>Посмотрим на распределение значений целевого параметра в общей выборке: **2229** значений **"0"** и **985 "1"**. Т.е. значение **"1"** составляет всего чуть больше **30%** всех значений целевого признака. При дальнейшем разделении имеет смысл воспользоваться параметром `stratify` для сохранения пропорции вариантов значений целевого параметра во всех выборках, полученных путем разделения. 

In [9]:
print(cell_comm['is_ultra'].value_counts())
print(str(985 * 100 / (2229 + 985)) + '%')

0    2229
1     985
Name: is_ultra, dtype: int64
30.647168637212197%


Ничего не "бросается в глаза".


Итак, в изучаемом датасете отсутствуют пропуски и дубликаты. Данные приведены к корректным типам. Можно приступать к дальнейшей работе.

## 2. Разбиение данных на выборки

Разделим наш датафрейм на две переменные: **features** - изменяемые признаки и **target** - целевой признак.

In [10]:
features = cell_comm.drop(['is_ultra'], axis=1)
target = cell_comm['is_ultra']

Запишем в переменную **rs** значение *777*, которое в дальнейших манипуляциях будем использовать в качестве значения атрибута `random_state`.

In [11]:
rs = 777

Разделим полученные переменные на 3 части каждую: тренировочная выборка `_train`, валидационная выборка `_valid` и тестовая выборка `_test` в соотношении **60:20:20**.

In [12]:
features_train, features_new, target_train, target_new = tts(features, target, test_size=.4,
                                                                random_state=rs, stratify=target)
features_valid, features_test, target_valid, target_test = tts(features_new, target_new,
                                                              test_size=.5, random_state=rs,
                                                              stratify=target_new)

Присвоим всем полученным в результате выполнения предыдущей операции датафреймам соответствующие имена.

In [13]:
features_train.name = 'features_train'
features_valid.name = 'features_valid'
features_test.name = 'features_test'
target_train.name = 'target_train'
target_valid.name = 'target_valid'
target_test.name = 'target_test'

Выведем на экран размерности всех полученных новых датафреймов (переменных, служащих для обучения модели, валидации и проверки результата).

Дополнительно выведем размерность исходного датафрейма.

In [14]:
data = [features_train, target_train, features_valid, target_valid,
       features_test, target_test]
print('cell_comm:', cell_comm.shape)
print('features:', features.shape)
print('target:', target.shape)

for i in data:
    print(i.name, i.shape)

cell_comm: (3214, 5)
features: (3214, 4)
target: (3214,)
features_train (1928, 4)
target_train (1928,)
features_valid (643, 4)
target_valid (643,)
features_test (643, 4)
target_test (643,)


Нам доступна выборка из 3214 строк и 5 столбцов.
Сперва мы разбили исходный датасет на признаки-переменные **features** и целевой признак **target**. Далее разделили генеральную выборку на обучающую (тренировочный набор данных) `_train` и проверочную `_new` в соотношении **60:40**. После чего проверочную выборку разделили в соотношении **50:50** на валидационную `_valid` и тестовую `_test`.

Таким образом мы получили разбивку данных на **3 выборки**: `_train`, `_valid` и `_test` в соотношении **60:20:20**. Выпоним простейшую проверку:

In [15]:
print(features.shape[0] * 0.6)
print(features.shape[0] * 0.2)

1928.3999999999999
642.8000000000001


Все совпало, тренировочная выборка содержит 1928 объектов, валидационная и тестовая выборки - по 643 объекта.

## 3. Исследование моделей

### 3.1. Обучение модели "дерево решений"

In [16]:
best_depth = 0
best_result = 0
# через цикл зададим значением аргумента максимальной глубины дерева диапазон от 1 до 10
for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(max_depth=depth, random_state=rs)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    # выявим наилучшую величину глубины
    if result > best_result:
        best_depth = depth
        best_result = result
        
print('Оптимальная глубина дерева:', best_depth)
print('Лучший результат accuracy:', best_result)

Оптимальная глубина дерева: 9
Лучший результат accuracy: 0.7869362363919129


<hr style="border: 2px solid orange;" />

### 3.2. Обучение модели "случайный лес"

In [18]:
best_depth = 0
best_est = 0
best_result = 0
# через цикл зададим значением аргумента количества оценщиков (деревьев) диапазон от 5 до 100
for est in range(5, 100, 10):                                                        #шаг 10
    # через цикл зададим значением аргумента максимальной глубины дерева диапазон от 1 до 5
    for depth in tqdm(range(1, 6), leave=False):
        model_forest = RandomForestClassifier(max_depth=depth, n_estimators=est,
                                             random_state=rs)
        model_forest.fit(features_train, target_train)
        predictions_valid = model_forest.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        # выявим наилучшую величину глубины дерева и количества оценщиков (деревьев)
        if result > best_result:
            best_depth = depth
            best_est = est
            best_result = result

In [19]:
print('Оптимальная глубина дерева:', best_depth)
print('Оптимальное количество деревьев:', best_est)
print('Лучший результат accuracy:', best_result)

Оптимальная глубина дерева: 5
Оптимальное количество деревьев: 35
Лучший результат accuracy: 0.7962674961119751


### 3.2. Обучение модели "логистическая регрессия"

In [20]:
model_logistic = LogisticRegression(random_state=rs, solver='liblinear')
model_logistic.fit(features_train, target_train)
predictions_valid = model_logistic.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)

print('Результат accuracy:', result)

Результат accuracy: 0.6982892690513219


Соберем полученные результаты вместе.

In [21]:
print('Результаты на валидационной выборке:')
print()
print('Дерево решений:', 0.79)
print('Случайный лес:', 0.79)
print('Логистическая регрессия:', 0.70)

Результаты на валидационной выборке:

Дерево решений: 0.8025
Случайный лес: 0.8025
Логистическая регрессия: 0.7185


Как мы видим, результаты **"случайного леса"** и **"дерева решений"** оказались равны **0.79**. Глубина **"дерева решений"** при этом составила **9** уровней, а **"случайный лес"** включает в себя **35 деревьев с глубиной 5**. У **"логистической регрессии"** лучший результат - **0.70**.

## 4. Проверка моделей на тестовой выборке

Для проверки моделей на тестовой выборке сперва объединим тренировочную и валидационную выборки воедино.

In [22]:
features_train_valid = pd.concat([features_train, features_valid])
target_train_valid = pd.concat([target_train, target_valid])

Выведем на экран размер обновленных выборок `features_train` и `target_train`, выполним проверку по моделям, обученным на выборке `_train` и отдельно по моделям, лбученным на `_train_valid`.

In [23]:
print('features_train_valid:', features_train_valid.shape)
print('target_train_valid:', target_train_valid.shape)
print(features.shape[0] * 0.8)

features_train_valid: (2571, 4)
target_train_valid: (2571,)
2571.2000000000003


### 4.1. Проверка моделей, обученных на изначальной выборке `_train`

In [24]:
# проверка модели "дерево решений"
model_tree = DecisionTreeClassifier(max_depth=8, random_state=rs)
model_tree.fit(features_train, target_train)
predictions_test_tree = model_tree.predict(features_test)
result_tree = accuracy_score(target_test, predictions_test_tree)

# проверка модели "случайный лес" 
model_forest = RandomForestClassifier(max_depth=5, n_estimators=55,random_state=rs)
model_forest.fit(features_train, target_train)
predictions_test_forest = model_forest.predict(features_test)
result_forest = accuracy_score(target_test, predictions_test_forest)

# проверка модели "логистическая регрессия"
model_logistic = LogisticRegression(random_state=rs, solver='liblinear')
model_logistic.fit(features_train, target_train)
predictions_test_logistic = model_logistic.predict(features_test)
result_logistic = accuracy_score(target_test, predictions_test_logistic)

print('Результаты на тестовой выборке:')
print()
print('Дерево решений:', result_tree)
print('Случайный лес:', result_forest)
print('Логистическая регрессия:', result_logistic)

Результаты на тестовой выборке:

Дерево решений: 0.7916018662519441
Случайный лес: 0.8087091757387247
Логистическая регрессия: 0.6998444790046656


### 4.2. Проверка моделей, обученных на измененной выборке `_train_valid`

In [27]:
# проверка модели "дерево решений"
model_tree = DecisionTreeClassifier(max_depth=8, random_state=rs)
model_tree.fit(features_train_valid, target_train_valid)
predictions_test_tree = model_tree.predict(features_test)
result_tree = accuracy_score(target_test, predictions_test_tree)

# проверка модели "случайный лес" 
model_forest = RandomForestClassifier(max_depth=5, n_estimators=55,random_state=rs)
model_forest.fit(features_train_valid, target_train_valid)
predictions_test_forest = model_forest.predict(features_test)
result_forest = accuracy_score(target_test, predictions_test_forest)

# проверка модели "логистическая регрессия"
model_logistic = LogisticRegression(random_state=rs, solver='liblinear')
model_logistic.fit(features_train_valid, target_train_valid)
predictions_test_logistic = model_logistic.predict(features_test)
result_logistic = accuracy_score(target_test, predictions_test_logistic)

print('Результаты на тестовой выборке:')
print()
print('Дерево решений:', result_tree)
print('Случайный лес:', result_forest)
print('Логистическая регрессия:', result_logistic)

Результаты на тестовой выборке:

Дерево решений: 0.8009331259720062
Случайный лес: 0.8055987558320373
Логистическая регрессия: 0.6982892690513219


По итогам проверки на тестовой выборке лучший результат показала модель типа **"случайный лес"** с результатом **0.81** (для модели, обученной на расширенной выборке этот результат составляет **0.806**). **"Дерево решений"** , показавшее аналогичный победителю результат по валидационной выборке, на тестовой выборке показал `accuracy` всего **0.79** (и **0.8** для расширенной обучающей выборки). 

**"Логистическая регрессия"** в обоих случаях показала результат **0.7**.

По итогам проверки на тестовой выборке побеждает **"случайный лес"** с глубиной дерева `max_depth`, равной **5** и количеством наблюдателей **35**.

## 5. Проверка модели на адекватность

Для проверки модели на адекватность воспользуемся фиктивным классификатором **DummyClassifier** с атрибутом `strategy = 'most_frequent'`. Т.е. результатом для каждого объекта будет наиболее часто встречающийся целевой признак.

In [26]:
model_dummy = DummyClassifier(strategy='most_frequent', random_state=rs)
model_dummy.fit(features_train, target_train)
predictions_valid_dummy = model_dummy.predict(features_valid)
predictions_test_dummy = model_dummy.predict(features_test)
result_valid_dummy = accuracy_score(target_valid, predictions_valid_dummy)
result_test_dummy = accuracy_score(target_test, predictions_test_dummy)

print('Результат по валидационной выборке:', result_valid_dummy)
print('Результат по тестовой выборке:', result_test_dummy)

Результат по валидационной выборке: 0.6936236391912908
Результат по тестовой выборке: 0.6936236391912908


Поскольку мы заполнили целевой признак наиболее часто встречающимся значением из 2 возможных, результат ожидаемо должен находиться в диапазоне **от 0.5 до 1.0**. Мы получили *по тестовой и валидацонной выборкам* значение `accuracy`, равное **0.69** при **0.81 для случайного леса** с оптимальными параметрами. Результаты по двум выборкам совпадают, поскольку в них одинаковое количество элементов, и соотношение значений целевого параметра выбрано пропорционально оному для общей выборки.

Таким образом, можем заключить, что в нашем случае модель типа **"случайный лес"** является *адекватной задаче и оптимальной на фоне других возможных моделей*.